In [1]:
import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

Intitializing Scala interpreter ...

Spark Web UI available at http://LAPTOP-0NFRQ5HD:4040
SparkContext available as 'sc' (version = 3.1.1, master = local[*], app id = local-1617951225622)
SparkSession available as 'spark'


import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD


In [103]:
val graph = GraphLoader.edgeListFile(sc,"data/soc-karate.mtx")
                .mapEdges(e => scala.util.Random.nextFloat)
                .mapVertices[String]((vid,i) =>  if(scala.util.Random.nextFloat <0.1) "infected" else "susceptible")

graph: org.apache.spark.graphx.Graph[String,Float] = org.apache.spark.graphx.impl.GraphImpl@44e8506f


In [104]:
graph.vertices.collect

res66: Array[(org.apache.spark.graphx.VertexId, String)] = Array((34,susceptible), (4,susceptible), (16,susceptible), (22,infected), (28,susceptible), (30,susceptible), (14,susceptible), (32,susceptible), (24,susceptible), (6,susceptible), (8,susceptible), (12,susceptible), (18,susceptible), (20,infected), (26,susceptible), (10,susceptible), (2,susceptible), (13,susceptible), (19,infected), (15,susceptible), (21,susceptible), (25,susceptible), (29,susceptible), (11,susceptible), (27,susceptible), (33,susceptible), (23,infected), (1,susceptible), (17,susceptible), (3,susceptible), (7,susceptible), (9,susceptible), (31,susceptible), (5,susceptible))


In [116]:
def mergeMsg (m1:String, m2:String):String = {

    if(m1=="become_inactive" && m2 == "become_inactive"){
        return "become_inactive"
    }
    if (m1 == "become_infected" || m2 == "become_infected"){
        return "become_infected"
    }
    return "nothing"
}

def vprog(VertexId:VertexId, VD:String, A:String): String ={
    if (VD == "susceptible" && A == "become_infected"){
        return "infected"
    }
    if(VD=="infected" && A == "become_inactive"){
        return "inactive"
    }
    return VD
}

def sendMsg(triplet:EdgeTriplet[String, Float]): Iterator[(VertexId, String)]={
    if (triplet.srcAttr == "infected"){
        if (scala.util.Random.nextFloat < triplet.attr){
            return Iterator((triplet.dstId,"become_infected"),(triplet.srcId,"become_inactive"))
        }
    return Iterator((triplet.srcId,"become_inactive"))
    }
    return Iterator()
  
}

mergeMsg: (m1: String, m2: String)String
vprog: (VertexId: org.apache.spark.graphx.VertexId, VD: String, A: String)String
sendMsg: (triplet: org.apache.spark.graphx.EdgeTriplet[String,Float])Iterator[(org.apache.spark.graphx.VertexId, String)]


In [117]:
graph.pregel("nothing",Int.MaxValue,EdgeDirection.Out)(vprog,sendMsg,mergeMsg)
    .mapVertices((vid,vdata) => if(vdata == "infected") "inactive" else vdata)
    .vertices.collect()

res76: Array[(org.apache.spark.graphx.VertexId, String)] = Array((34,susceptible), (4,susceptible), (16,susceptible), (22,inactive), (28,susceptible), (30,susceptible), (14,susceptible), (32,susceptible), (24,susceptible), (6,susceptible), (8,susceptible), (12,susceptible), (18,susceptible), (20,inactive), (26,susceptible), (10,susceptible), (2,inactive), (13,susceptible), (19,inactive), (15,susceptible), (21,susceptible), (25,susceptible), (29,susceptible), (11,susceptible), (27,susceptible), (33,susceptible), (23,inactive), (1,inactive), (17,susceptible), (3,susceptible), (7,susceptible), (9,susceptible), (31,susceptible), (5,susceptible))


In [110]:
graph.triplets.filter(t => t.dstId == 23).collect

res71: Array[org.apache.spark.graphx.EdgeTriplet[String,Float]] = Array(((33,susceptible),(23,infected),0.9540973), ((34,susceptible),(23,infected),0.3116812))
